# 引入Header

In [1]:
from  scapy.all import *
import netifaces
import datetime
import ipaddress
import pandas as pd
from subprocess import run

# 擷取封包
### 利用 ARP 及SSDP 封包 (需要100個有效的封包，最多蒐集30秒)
### 並定義data num arp 分別儲存字串類型的IP 位址、數字類型的IP 位址及從ARP蒐集過來的IP位址

In [ ]:
catch =sniff (iface='eth0',filter ="ip and udp port 1900 or arp" ,count=100,timeout=30)
lenth=len(catch)
data=[]
num=[]
arp=[]
print (catch)

# Function 過濾正確的IP位址

In [ ]:
def check_ip (ip_addr):
    if (ip_addr!="0.0.0.0" and ip_addr!="255.255.255.255"):
        tmp=int(ipaddress.IPv4Address(ip_addr))
        if (tmp<=3758096384): #CLASS D,E
            if (tmp>2851995648 and tmp<2852061183): #169.254.0.0/16
                return False
            else:
                return True
        else:
            return False
    else:
        return False


# 儲存資料
### type 2048 為一般的封包
### type 2054 則為ARP

In [ ]:
for i in range(lenth):
    if (int(catch[i][0].type)==2048):
        if (check_ip(catch[i][IP].src)==True):
            tmp=catch[i][IP].src
            data.append(tmp)
            num.append(int(ipaddress.ip_address(tmp)))
    elif (int(catch[i][0].type)==2054):
        if (check_ip(catch[i][ARP].psrc)==True):
            tmp=catch[i][ARP].psrc
            arp.append(tmp)
            data.append(tmp)
            num.append(int(ipaddress.ip_address(tmp)))
        if (check_ip(catch[i][ARP].pdst)==True):
            dst=catch[i][ARP].pdst
            arp.append(dst)

# data 及num 轉為DataFrame

In [ ]:
data={'number':num,'ip_address':data}
data=pd.DataFrame(data)
data=data.drop_duplicates('ip_address',keep='first',inplace=False)
data=data.sort_values(by='number')
print (data)

# 計算 網路遮罩

In [ ]:
last=(data.iloc[-2]).number
print (last)
first=(data.iloc[1]).number
print (first)
mask_fl=((last&first)|(~(last|first)))
mask_c=mask_fl+256**4
mask_c=str(ipaddress.IPv4Address(int (mask_c)))
mask=0
for i in range(2,31):
    tmp="255.255.255.255/" + str(i)
    tmp=ipaddress.ip_network(tmp,strict=False)
    if(ipaddress.IPv4Address(mask_c)<tmp.netmask):
        print (i)
        mask=i-1
        break
mask_str=ipaddress.ip_network("255.255.255.255/" + str(mask),strict=False)

# 計算 Gateway

In [ ]:
dict={}
maxaddr=0
GW=''
for key in arp:
    dict[key]=dict.get(key,0)+1
    if (maxaddr<dict[key]):
        maxaddr=dict[key]
        GW=key
print(dict)

# 尋找可用之IP

In [ ]:
for i in range (0,len(data)-1):
    ip=0
    if (((data.iloc[i]).number-(data.iloc[i+1]).number)<1):
        ip=((data.iloc[i]).number)+1
        break


# 輸出計算結果

In [ ]:
print("Result")
print ("IP address:",end=" ")
print (ipaddress.ip_address(int (ip)))
print ("Network mask:",end=" ")
print(mask_str.netmask)
print ("Gateway:",end=" ")
print (GW)

# 直接輸出DHCP結果

In [ ]:
addr=netifaces.ifaddresses('eth0')[2][0]['addr']
netmask=netifaces.ifaddresses('eth0')[2][0]['netmask']
gateway=netifaces.gateways()[2][0][0]
network_inf = "Address: %s\nNetmask: %s\nGateway: %s\n"%(addr,netmask,gateway)
print (network_inf)